In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

import cv2

import time

In [2]:
from main import open_game
import image_parser

In [3]:
driver = open_game()

In [ ]:
time.sleep(10)

In [4]:
iframe = driver.find_element(by=By.ID, value='game_drop')
height = iframe.rect["height"]  # 540
width = iframe.rect["width"]  # 960

In [5]:
iframe.click()
iframe.click()

In [6]:
# type name
ActionChains(driver).move_to_element_with_offset(
    iframe, 0, int(0.1 * height)
).click().send_keys("selenium").perform()

In [7]:
# start game
ActionChains(driver).move_to_element_with_offset(
    iframe, 0, int(0.2 * height)
).click().perform()

In [8]:
# skip tutorial
ActionChains(driver).move_to_element_with_offset(
    iframe, -int(0.35 * width), int(0.4 * height)
).click().perform()

In [9]:
# turn off sprint animation
ActionChains(driver).move_to_element_with_offset(
    iframe, -int(0.45 * width), -int(0.42 * height)  # move to settings icon
).click().move_to_element_with_offset(
    iframe, int(0.1 * width), int(0.07 * height)  # move to animation checkbox
).click().move_to_element_with_offset(
    iframe, -int(0.45 * width), -int(0.42 * height)  # move to settings icon
).click().perform()

In [10]:
# buy research
ActionChains(driver).move_to_element_with_offset(
    iframe, int(0.3 * width), -int(0.3 * height)
).click().perform()

In [10]:
iframe.screenshot('iframe.png')

True

In [3]:
image = cv2.imread('iframe.png')
meta_info = image[7:83, 57:932]

In [4]:
sprint_n = image_parser.get_sprint_number(meta_info)
sprint_n

4.0

In [5]:
image_parser.get_game_money(meta_info)

33000.0

In [6]:
image_parser.get_customers(meta_info)

25.0

In [7]:
image_parser.get_loyalty(meta_info)

4.0

In [8]:
credit = max(300_000 - (sprint_n - 1) * 9_000, 0)
credit

273000.0

In [9]:
available_developers_count = 2
current_rooms_counter = 1
blank_sprint_counter = 0

In [11]:
current_sprint_hours = image_parser.get_current_sprint_hours(image)
current_sprint_hours

0.0

In [12]:
user_stories = image_parser.get_user_stories(image)
user_stories

[(array([115, 188,  30], dtype=uint8), 0.055, 2.0)]

In [13]:
iframe.rect

{'height': 540, 'width': 960, 'x': 29.600000381469727, 'y': 20}

In [25]:
driver.quit()